<a href="https://colab.research.google.com/github/abinhack/ds_experiments/blob/main/Pytorch_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [ ]:
class NN(nn.Module):
  def __init__(self,input_size,num_classes):
    super(NN,self).__init__()
    self.fc1 = nn.Linear(input_size,50)
    self.fc2 = nn.Linear(50,num_classes)
  def forward(self,x):
    x =F.relu(self.fc1(x))
    x = self.fc2(x)
    return x


model = NN(784,10)
x = torch.randn(64,784)
print(model(x).shape)

torch.Size([64, 10])


In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
input_size=784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10

In [ ]:

train_dataset = datasets.MNIST(root='dataset/',train=True,transform=transforms.ToTensor(),download=True)
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_dataset = datasets.MNIST(root='dataset/',train=False,transform=transforms.ToTensor(),download=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw




Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw




Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
model = NN(input_size=input_size,num_classes=num_classes).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer =optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
%timeit
for epoch in range(num_epochs):
  for batch_idx, (data,targets) in enumerate(train_loader):
    data = data.to(device=device)
    targets = targets.to(device=device)

    #Get the correct shape
    data =data.reshape(data.shape[0],-1)
    #forward
    scores = model(data)
    loss = criterion(scores,targets)

    #backward
    optimizer.zero_grad()
    loss.backward()
    #gradient descent or adam setp
    optimizer.step()

def check_accuracy(loader,model):
  num_correct =0 
  num_samples =0
  model.eval()
  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      x = x.reshape(x.shape[0],-1)
      scores = model(x)
      _, predictions =scores.max(1)
      num_correct += (predictions==y).sum()
      num_samples += predictions.size(0)
    print(f'Got{num_correct}/{num_samples} with accuracy {num_correct/num_samples}')
  model.train()
  #return acc

check_accuracy(train_loader,model)
check_accuracy(test_loader,model)



Got59101/60000 with accuracy 0.9850167036056519
Got9687/10000 with accuracy 0.9686999917030334
